In [12]:
import pandas as pd
import numpy
import torch 
import numpy as np
from torch.utils.data import Dataset, DataLoader
import sys
sys.path.append("..")
from utils import pad_collate
from dataloader_comma import CommaDataset
from dataloader_nuscenes import NUScenesDataset
from collections import Counter
from model import VTN
import matplotlib.pyplot as plt 
from PIL import Image
import glob
import os
from utils import * 
import re
from vis_utils import * 
import json


In [50]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')

stop_words = set(stopwords.words('english'))  # Use the English stopwords list

def remove_stopwords(sentence):
    tokenized_words = sentence.split()
    filtered_words = [word for word in tokenized_words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jessica/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [83]:
alle = []
for j, batch in enumerate(dataloader_nuscenes):
    scene_name, image_array, vego, angle, distance = batch
    image_array, vego, angle, distance = image_array[0:20], vego[0:20], angle[0:20], distance[0:20]
    img = image_array
    img = img.to(f'cuda:{gpu_num}')
    angle = angle.to(f'cuda:{gpu_num}')
    distance = distance.to(f'cuda:{gpu_num}')
    vego = vego.to(f'cuda:{gpu_num}')
    (logits, attns), concepts = model(img, angle, distance, vego)
    top5_indices = torch.tensor(concepts.squeeze()).topk(3).indices
    s = img.shape#[batch_size, seq_len, h,w,c]
    angle = angle.to("cpu")
    distance = distance.to("cpu")
    vego = vego.to("cpu")
    logits = logits.detach().cpu().to("cpu")
    concepts = concepts.detach().cpu().to("cpu")
    concepts_preds = []
    for i, elem in enumerate(top5_indices):
        for a in elem: 
            concepts_preds.append(scenarios[a.item()])
    count_dict = Counter(concepts_preds)
    top_5 = count_dict.most_common(3)
    real_scen = remove_stopwords(scenarios_scenes[scene_name[0]].lower()).replace('peds', 'pedestrian')
    real_scens = real_scen.split(",")

    for scen in real_scens:
        res = False
        for elem in top_5:
            res = False
            k = remove_stopwords(elem[0].replace('peds', 'pedestrian'))
            scen_list = np.array(k.lower().split(' '))
            uni = np.unique(scen_list)
            compare = " ".join(uni)
            for word in scen.split(" "):
                if word in compare: 
                    res = True
                    break
                res = False

        alle.append(res)

/tmp/ipykernel_3720037/4061143721.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  top5_indices = torch.tensor(concepts.squeeze()).topk(3).indices


In [84]:
np.array(alle).mean()

0.8145161290322581

In [14]:
f = open('/data1/jessica/data/toyota/nuscenes/scenes_nuscenes.json')
scenarios_scenes = json.load(f)

In [28]:
scenarios_scenes

{'scene-0001': 'Construction, maneuver between several trucks',
 'scene-0002': 'Intersection, peds, waiting vehicle, parked motorcycle at parking lot',
 'scene-0003': 'Parking lot, barrier, exit parking lot',
 'scene-0004': 'Random scene, arrive at intersection, cross intersection, jaywalkers',
 'scene-0005': 'Overtaken by taxi, construction site',
 'scene-0006': 'Sitting construction worker, parked bicycle, ped walking along road, construction vehicle, worker sitting on back of van',
 'scene-0007': 'Van overtaking us, peds, jaywalker, bicycle rack, turn left',
 'scene-0008': 'Several moving pedestrians, parked cards and motorcycle',
 'scene-0009': 'Peds crossing, parked cars, garbage, peds waiting',
 'scene-0010': 'Parked motorcycle, person pickung up garbage, pass construction vehicles and entrance to construction site',
 'scene-0011': 'Turn left, pass trucks on both sides, construction workers, sleeping construction workers, garbage, parked cars',
 'scene-0012': 'Nature, bus stop, p

In [27]:
for elem in range(len(scene_name)):
    print(scenarios_scenes[scene_name[elem]])

Pedestrian crossing, stopped vehicle


In [16]:
len(concepts_preds)

207